In [3]:
!pip install catboost

   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB 640.0 kB/s eta 0:02:39
   ---------------------------------------- 0.1/101.7 MB 762.6 kB/s eta 0:02:14
   ---------------------------------------- 0.1/101.7 MB 944.1 kB/s eta 0:01:48
   ---------------------------------------- 0.2/101.7 MB 1.1 MB/s eta 0:01:29
   ---------------------------------------- 0.3/101.7 MB 1.3 MB/s eta 0:01:21
   ---------------------------------------- 0.4/101.7 MB 1.3 MB/s eta 0:01:18
   ---------------------------------------- 0.4/101.7 MB 1.3 MB/s eta 0:01:18
   ---------------------------------------- 0.5/101.7 MB 1.4 MB/s eta 0:01:12
   ---------------------------------------- 0.6/101.7 MB 1.4 MB/s eta 0:01:15
   ---------------------------------------- 0.7/101.7 MB 1.4 MB/s eta 0:01:13
   ---------------------------------------- 0.7/101.7 MB 1.4 MB/s eta 0:

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [7]:
train_dir = r'D:\Term1th4\Computer Vision\Sections\SEC4\Alzheimer_s Dataset\Alzheimer_s Dataset\train'  
test_dir = r'D:\Term1th4\Computer Vision\Sections\SEC4\Alzheimer_s Dataset\Alzheimer_s Dataset\test'

In [9]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [11]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [13]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [15]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [17]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\Taha Tarek\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

C:\Users\Taha Tarek\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 9s 58ms/step


In [21]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step


In [23]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3273788	total: 174ms	remaining: 2m 53s
1:	learn: 1.2750914	total: 198ms	remaining: 1m 38s
2:	learn: 1.2313708	total: 224ms	remaining: 1m 14s
3:	learn: 1.1945935	total: 249ms	remaining: 1m 1s
4:	learn: 1.1608544	total: 273ms	remaining: 54.4s
5:	learn: 1.1306035	total: 297ms	remaining: 49.2s
6:	learn: 1.1059975	total: 321ms	remaining: 45.5s
7:	learn: 1.0830516	total: 345ms	remaining: 42.8s
8:	learn: 1.0634363	total: 367ms	remaining: 40.4s
9:	learn: 1.0456507	total: 389ms	remaining: 38.5s
10:	learn: 1.0291938	total: 417ms	remaining: 37.5s
11:	learn: 1.0116341	total: 444ms	remaining: 36.5s
12:	learn: 0.9978651	total: 468ms	remaining: 35.5s
13:	learn: 0.9863193	total: 489ms	remaining: 34.5s
14:	learn: 0.9761349	total: 510ms	remaining: 33.5s
15:	learn: 0.9667845	total: 532ms	remaining: 32.7s
16:	learn: 0.9564426	total: 556ms	remaining: 32.2s
17:	learn: 0.9482555	total: 578ms	remaining: 31.6s
18:	learn: 0.9398022	total: 604ms	remaining: 31.2s
19:	learn: 0.9317102	total: 635ms	rema

In [25]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [27]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 57.31%


In [29]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [31]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [33]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model('cnn_feature_extractor6.h5')
catboost_classifier = joblib.load('catboost_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = r'D:\Term1th4\Computer Vision\Sections\SEC4\Alzheimer_s Dataset\Alzheimer_s Dataset\test\NonDemented\26 (63).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
The image is classified as: NonDemented


C:\Users\Taha Tarek\AppData\Local\Temp\ipykernel_14832\3555952806.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(prediction[0])
